In [ ]:
import pandas as pd

In [ ]:
terminal_df = pd.read_csv('../data/raw/terminal.csv').fillna('')

In [ ]:
# terminal_df.head()

In [ ]:
terminal_df[['terminal_facility_name','terminal_company_name']] = terminal_df[['terminal_facility_name','terminal_company_name']].astype(str).apply(lambda x: x.str.upper())

In [ ]:
# terminal_df.head()

In [ ]:
terminal_df[['terminal_lat','terminal_lon']] = terminal_df[['terminal_lat','terminal_lon']].astype(str).apply(lambda x: x.str.replace(' ', ''))

In [ ]:
terminal_df.head()

In [ ]:
terminal_df[['terminal_lat','terminal_lon']] = terminal_df[['terminal_lat','terminal_lon']].astype(str).apply(lambda x: x.str[1:] + x.str[0])

In [ ]:
terminal_df.head()

In [ ]:
"""
Converting Degrees, Minutes, Seconds formatted coordinate strings to decimal. 

Formula:
DEC = (DEG + (MIN * 1/60) + (SEC * 1/60 * 1/60))

Assumes S/W are negative. 
"""

import re

#https://www.javaer101.com/en/article/18832510.html
#https://stackoverflow.com/questions/33997361
def dms2dd(s):
    # example: s = """0°51'56.29"S"""
    degrees, minutes, seconds, direction = re.split('[°\'"]+', s)
    dd = float(degrees) + float(minutes)/60 + float(seconds)/(60*60);
    if direction in ('S','W'):
        dd*= -1
    return dd

In [ ]:
terminal_df['terminal_lat'] = terminal_df['terminal_lat'].apply(dms2dd)
terminal_df['terminal_lon'] = terminal_df['terminal_lon'].apply(dms2dd)

In [ ]:
terminal_df.to_json ('../data/processed/terminal.json', orient='records', indent=2)

In [ ]:
terminal_df

In [ ]:
tac1_df = pd.read_csv('../data/raw/TAC1_service_proforma.csv').fillna('')

In [ ]:
tac1_df

In [ ]:
import networkx as nx

In [ ]:
edge_columns = ['service_id_fk', 'service_trade', 'service_trade_lane', 'transport_edge_no', 'transport_call_port_1', 'service_proforma_port_call_sequence_no_port1', 'service_proforma_terminal_call_sequence_no_port1', 'transport_call_port_2', 'service_proforma_port_call_sequence_no_port2', 'service_proforma_terminal_call_sequence_no_port2']

In [ ]:
tac_edges_df = pd.DataFrame(columns = edge_columns)

In [ ]:
tac_edges_df

In [ ]:
curr_row = None
prev_row = None
num = -1

for index, row in tac1_df.iterrows():
    if all(v is not None for v in [curr_row, prev_row]):
        if curr_row['service_trade_lane']==prev_row['service_trade_lane']:
#             print(str(num), curr_row['terminal'], prev_row['terminal'])
            tac_edges_df = tac_edges_df.append({'service_id_fk': prev_row['service_id_fk'],
                                                'service_trade': prev_row['service_trade'],
                                                'service_trade_lane': prev_row['service_trade_lane'],
                                                'transport_edge_no': prev_row['service_id_fk'] + '_' + str(num),
                                                'transport_call_port_1': prev_row['terminal'],
                                                'service_proforma_port_call_sequence_no_port1': prev_row['service_proforma_port_call_sequence_no'],
                                                'service_proforma_terminal_call_sequence_no_port1': prev_row['service_proforma_terminal_call_sequence_no'],
                                                'transport_call_port_2': curr_row['terminal'],
                                                'service_proforma_port_call_sequence_no_port2': curr_row['service_proforma_port_call_sequence_no'],
                                                'service_proforma_terminal_call_sequence_no_port2': curr_row['service_proforma_terminal_call_sequence_no']
                                               }, 
                                               ignore_index = True)
        prev_row = curr_row;
        
    if prev_row is None and curr_row is not None:
#         print(curr_row)
        prev_row = curr_row
        
    curr_row = row;
    num += 1
    
if curr_row['service_trade_lane']==prev_row['service_trade_lane']:
#             print(str(num), curr_row['terminal'], prev_row['terminal'])
            tac_edges_df = tac_edges_df.append({'service_id_fk': prev_row['service_id_fk'],
                                                'service_trade': prev_row['service_trade'],
                                                'service_trade_lane': prev_row['service_trade_lane'],
                                                'transport_edge_no': prev_row['service_id_fk'] + '_' + str(num),
                                                'transport_call_port_1': prev_row['terminal'],
                                                'service_proforma_port_call_sequence_no_port1': prev_row['service_proforma_port_call_sequence_no'],
                                                'service_proforma_terminal_call_sequence_no_port1': prev_row['service_proforma_terminal_call_sequence_no'],
                                                'transport_call_port_2': curr_row['terminal'],
                                                'service_proforma_port_call_sequence_no_port2': curr_row['service_proforma_port_call_sequence_no'],
                                                'service_proforma_terminal_call_sequence_no_port2': curr_row['service_proforma_terminal_call_sequence_no']
                                               }, 
                                               ignore_index = True)

In [ ]:
tac_edges_df

In [ ]:
tac_edges_df.to_json ('../data/processed/edges.json', orient='records', indent=2)